In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/12 05:01:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os

file_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/home_sales_revised.csv"
print(os.path.exists(file_path))  # This should return True if the file exists

True


In [8]:
# Check if the file exists
if os.path.exists(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Create a temporary view of the DataFrame
        df.createOrReplaceTempView("home_sales")
        
        # Display the DataFrame to confirm it loaded correctly
        df.show()
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
else:
    print(f"The file does not exist at the specified path: {file_path}")

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
average_price_query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

average_price_df = spark.sql(average_price_query)
average_price_df.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [12]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
average_price_built_query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    bedrooms = 3 AND bathrooms = 3 
GROUP BY 
    date_built 
ORDER BY 
    year_built
"""

# Run the query and create a DataFrame
average_price_built_df = spark.sql(average_price_built_query)

# Show the results
average_price_built_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
average_price_built_query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    bedrooms = 3 
    AND bathrooms = 3 
    AND floors = 2 
    AND sqft_living >= 2000 
GROUP BY 
    date_built 
ORDER BY 
    year_built
"""

# Run the query and create a DataFrame
average_price_built_df = spark.sql(average_price_built_query)

# Show the results
average_price_built_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [16]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Start the timer
start_time = time.time()

# SQL query to calculate the average price of homes per view rating
average_price_view_query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    price >= 350000 
GROUP BY 
    view 
ORDER BY 
    view DESC
"""

# Run the query and create a DataFrame
average_price_view_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_df.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.0195181369781494 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

average_price_view_query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    price >= 350000 
GROUP BY 
    view 
ORDER BY 
    view DESC
"""

# Start the timer for the cached query
start_time = time.time()

# Run the query using the cached data and create a DataFrame
average_price_view_cached_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_cached_df.show()

# Calculate and print the runtime for the cached query
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.710698127746582 seconds ---


In [30]:
# 11. Read the formatted parquet data.
# Define the output path for the partitioned Parquet data
output_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/home_sales_partitioned/date_built=__HIVE_DEFAULT_PARTITION__/part-00000-dd84f69f-e405-4b20-87e6-9b5bf2c96b0d.c000.snappy.parquet"

df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

In [32]:
# 11. Read the formatted parquet data.
# Define the output path for the partitioned Parquet data
output_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/home_sales_partitioned/date_built=__HIVE_DEFAULT_PARTITION__/part-00000-dd84f69f-e405-4b20-87e6-9b5bf2c96b0d.c000.snappy.parquet"

# Read the formatted Parquet data back into a DataFrame
df_parquet = spark.read.parquet(output_path)

# Show the DataFrame to verify the data
df_parquet.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [34]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales")

In [36]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
# Start the timer
start_time = time.time()

# Run the SQL query
average_price_query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY average_price DESC
"""

# Execute the query
average_price_df = spark.sql(average_price_query)

# Show the results
average_price_df.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 1.5248041152954102 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [40]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' cached? {is_cached}")

Is 'home_sales' cached? False
